# Purpose
This notebooks adapts the weights scrapped online to my implementation architecture.

# Import

In [1]:
import torch
from glic.networks.completion_network import CompletionNetwork

# Compare the architectures

In [2]:
# my implementation CN architecure
my_cn = CompletionNetwork()
my_cn_state_dict = my_cn.state_dict()

In [3]:
# # download the weight
# trained_cn_weights_address = "http://iizuka.cs.tsukuba.ac.jp/data/completionnet_places2.t7"
# import wget
# wget.download(trained_cn_weights_address,"../logs/scrapped_weights/trained_cn_weights.t7")

Weights were converted using https://github.com/clcarwin/convert_torch_to_pytorch  
*The github project had to be forked to account for SpacialDilatedConvolution*

In [4]:
# load the converted weights
scrapped_weights = torch.load("../logs/scrapped_weights/trained_cn_weights.pth")

# Convert
Map by hand !

In [17]:
def get_next_key(ordered_dict,key):
    """This function returns the next key in the ordered_dict."""
    next = list(ordered_dict.keys())[list(ordered_dict.keys()).index(key) + 1]
    return next

In [18]:
from collections import OrderedDict

# prime
map_scrapped_to_my_cn = OrderedDict()
current_my_cn_key = list(my_cn_state_dict.keys())[0]

# iterative
for key in scrapped_weights:
    next_key = get_next_key(my_cn_state_dict,current_my_cn_key)
    # compare to next layer
    if scrapped_weights[key].shape == my_cn_state_dict[next_key]:
        map_scrapped_to_my_cn[key] = next_key
        current_my_cn_key = next_key
    else:
        map_scrapped_to_my_cn[key] = current_my_cn_key